# Kaggle Predicting Future Sales

Objective: minimise RMSE
<b>NOTE</b> True target values are clipped into [0,20]!!

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as graph
import seaborn as sns
import altair as alt

from tqdm import tqdm
from IPython.display import display

graph.style.use('fivethirtyeight')

# Load Data

In [ ]:
!wc -l data/competitive-data-science-predict-future-sales/*csv

In [ ]:
%%time
df_test = pd.read_csv('data/competitive-data-science-predict-future-sales/test.csv')
print(df_test.shape)
display(df_test.head())

In [ ]:
%%time
df_items = pd.read_csv('data/competitive-data-science-predict-future-sales/items.csv')
print(df_items.shape)
display(df_items.head())

In [ ]:
%%time
df_item_cat = pd.read_csv('data/competitive-data-science-predict-future-sales/item_categories.csv')
print(df_item_cat.shape)
display(df_item_cat.head())

In [ ]:
%%time
df_shops = pd.read_csv('data/competitive-data-science-predict-future-sales/shops.csv')
print(df_shops.shape)
display(df_shops.head())

In [ ]:
%%time
df_sales = pd.read_csv('data/competitive-data-science-predict-future-sales/sales_train.csv')
df_sales = df_sales.merge(df_items[['item_id', 'item_category_id']], on='item_id')

print(df_sales.shape)
display(df_sales.head())

In [ ]:
a = set(df_sales['shop_id'].astype(str) + '|' + df_sales['item_id'].astype(str))
b = set(df_test['shop_id'].astype(str) + '|' + df_test['item_id'].astype(str))

In [ ]:
print(len(a))
print(len(b))
print('intersection', len(a & b))
print('union', len(a | b))

<b>Sales Metadata</b>

the training set. Daily historical data from January 2013 to October 2015.

1. ID - an Id that represents a (Shop, Item) tuple within the test set
2. shop_id - unique identifier of a shop
3. item_id - unique identifier of a product
3. item_category_id - unique identifier of item category
4. <b>item_cnt_day</b> - number of products sold. <b><i>You are predicting a <u>monthly</u> amount of this measure</i></b>
5. item_price - current price of an item
6. date - date in format dd/mm/yyyy
7. date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
8. item_name - name of item
9. shop_name - name of shop
10. item_category_name - name of item category

# EDA and processing the data

In [ ]:
for col in ['shop_id', 'item_id', 'item_category_id']:
    print(f'N {col}:', df_sales[col].unique().shape)

In [ ]:
sns.distplot(df_sales['item_cnt_day'], kde=False)
graph.yscale('log')
graph.show()

graph.figure(figsize=(8, 4))
sns.distplot(df_sales['item_price'])
graph.xscale('log')
graph.show()

for col in ['shop_id', 'item_category_id']:
    graph.figure(figsize=(8, 4))
    graph.title(col.replace('_', ' ').title())
    for thing in tqdm(df_sales[col].unique()):
        sns.distplot(df_sales.query(f'{col} == {thing}')['item_price'], hist=False)
    graph.xlabel('Price ($)')
    graph.xscale('log')
    graph.show()

What the fuck exactly am I trying to predict??

In [ ]:
timeseries = df_sales.groupby('date_block_num')['item_cnt_day'].sum()
print(timeseries.shape)

graph.figure(figsize=(12, 5))
graph.plot(timeseries)
graph.xlabel('Date Block Number')
graph.show()

graph.figure(figsize=(12, 5))
for name, shop in df_sales.groupby('shop_id'):
    timeseries_i = shop.groupby('date_block_num')['item_cnt_day'].sum()
    graph.plot(timeseries_i, label=name, alpha=0.5, linewidth=1.2)
graph.xlabel('Date Block Number')
graph.show()

In [ ]:
%%time
# Create ML monthly sales dataframe
sales = df_test.copy()
sales.drop(columns='ID', inplace=True)
display(sales.head())
print(sales.shape)

expected_length = sales.shape[0] * len(timeseries)
print(expected_length)

# Repeat for every date block
subframes = []
for date_block in tqdm(sorted(df_sales['date_block_num'].unique()), desc='Repeating for all blocks'):
    frame_i = sales.copy()
    frame_i['date_block_num'] = date_block
    subframes.append(frame_i)
sales = pd.concat(subframes)
print(sales.shape)

assert len(sales) == expected_length, 'Concatenation failed!'
display(sales.sample(5))

# Summarized data into date blocks (date_block_num)
agg_sales = pd.pivot_table(
    df_sales, 
    values='item_cnt_day', 
    index=['shop_id', 'item_id', 'date_block_num'], 
    aggfunc=np.sum
)
agg_sales.reset_index(['shop_id', 'item_id', 'date_block_num'], inplace=True)
agg_sales.rename(columns={'item_cnt_day': 'item_cnt_month'}, inplace=True)
display(agg_sales.sample(5))
print(agg_sales.shape, 'shape or agg_sales (NOT final df)')

# Combine all of the data
sales = sales.merge(agg_sales, how='outer', on=['shop_id', 'item_id', 'date_block_num'])
display(sales.head())
print(sales.shape, 'after agg sales added')
print(sales.dropna().shape, 'no NaNs')

sales = sales.merge(df_items[['item_id', 'item_category_id']], how='left', on='item_id')
display(sales.head())
display(sales.tail())
print(sales.shape, 'after item category added')
print(sales.dropna().shape, 'no NaNs')

# Fillna with 0 because Nan means that month that shop did not sell that item
sales.fillna(0, inplace=True)
print(sales.shape)

# Baseline

Performance to beat the saying that last date block predicts the current

In [ ]:
baseline_sales = sales.query(f'date_block_num == {df_sales.date_block_num.max()-1}')
baseline_test_sales = sales.query(f'date_block_num == {df_sales.date_block_num.max()}')

print(baseline_sales.shape, baseline_test_sales.shape)

In [ ]:
def process_baseline(df):
    pivot = pd.pivot_table(
        df,
        values='item_cnt_month', index=['shop_id', 'item_id'],
        aggfunc=np.sum
    )
    pivot.reset_index(['shop_id', 'item_id'], inplace=True)
    return pivot

pivot_sales = process_baseline(baseline_sales)
pivot_test_sales = process_baseline(baseline_test_sales)

display(pivot_sales.sample(5))
display(pivot_test_sales.sample(5))

In [ ]:
# "Predict"
pivot_sales.rename(columns={'item_cnt_month': 'pred'}, inplace=True)
pivot_test_sales = pivot_test_sales.merge(pivot_sales, on=['shop_id', 'item_id'])

# Compute RMSE
pivot_test_sales['diff'] = pivot_test_sales['pred'] - pivot_test_sales['item_cnt_month']

display(pivot_test_sales.head())

In [ ]:
baseline_rmse = np.sqrt((pivot_test_sales['diff'] ** 2).mean())
print(f'Baseline RMSE = {baseline_rmse}')

# Predictive Model Making

Predict block `date block` + 1 from `(shop_id (one_hot), item_id (one_hot), date_block)`

## Remember
- Objective: minimise RMSE
- Target is to get a RMSE < 1.0
- True target values are clipped into [0,20]!!

In [ ]:
%%time
# Convert sales to x y data
data = sales.copy()
data.head()

In [ ]:
# TODO Create modulo 12 of the date_block to get the month of the year
y = data.pop('item_cnt_month')
x = data

graph.title('Unclipped Target')
sns.distplot(np.sqrt(y[y > 0] + 1e-9), kde=False)
graph.yscale('log')
graph.show()

graph.title('Clipped Target')
sns.distplot(np.clip(np.sqrt(y[y > 0]), 0, 20), kde=False)
graph.yscale('log')
graph.show()

print(y.shape, x.shape)
display(x.sample(5), y.sample(5))

In [ ]:
%%time
x_shop_1hot = pd.get_dummies(x['shop_id'], prefix='shop_id', drop_first=True)
display(x_shop_1hot.head())

In [ ]:
# XGBoost Model